In [1]:
import os 
import re
import matplotlib.pyplot as plt
import rmgpy.chemkin
import numpy as np
import cantera as ct
import random
import pandas as pd
from rmgpy.chemkin import load_chemkin_file
%matplotlib inline

Flipping thermos of each species one by one, siddhas CH3F model (her library, RMG thermo) to and siddhas other CH3F model (her library, her thermo)

In [2]:
#load the chemkin functions
def compare_species(model_1_species_list, model_2_species_list):
    """ Compares species list between models. """
    
    #two lists of same species, just have different names in each model
    common_species_1 = []
    common_species_2 = []
    
    #combined list of common species, with tuples of the two different names
    common_species = []
    
    for spc_1 in model_1_species_list:
        for spc_2 in model_2_species_list:
            if spc_1.is_isomorphic(spc_2):
                common_species_1.append(spc_1)
                common_species_2.append(spc_2)
                common_species.append((spc_1, spc_2))
                
                
    unique_species_1 = [spc_1 for spc_1 in model_1_species_list if spc_1 not in common_species_1]
    unique_species_2 = [spc_2 for spc_2 in model_2_species_list if spc_2 not in common_species_2]
    print(f'{len(common_species)} common species')


                
    return common_species, unique_species_1, unique_species_2

In [3]:
#full path
full_path = '/work/westgroup/nora/Code/'

#Siddha's CH3F model (new kinetic lib, RMG thermo in original order) #107 species
siddhas_chmkn = full_path + 'projects/PFAS/models/testing_Siddhas_chemistry/CH3F/Siddha_thermos_original_order/chemkin/chem_annotated.inp'
siddhas_transport = full_path + 'projects/PFAS/models/testing_Siddhas_chemistry/CH3F/Siddha_thermos_original_order/chemkin/tran.dat'
siddhas_species = full_path + 'projects/PFAS/models/testing_Siddhas_chemistry/CH3F/Siddha_thermos_original_order/chemkin/species_dictionary.txt'

#Siddha's CH3F model (new kinetic lib, new thermo) #104 species
siddhas_chmkn_with_thermo = full_path + 'projects/PFAS/models/testing_Siddhas_chemistry/CH3F/with_Siddhas_thermo/chemkin/chem_annotated.inp'
siddhas_transport_with_thermo = full_path + 'projects/PFAS/models/testing_Siddhas_chemistry/CH3F/with_Siddhas_thermo/chemkin/tran.dat'
siddhas_species_with_thermo = full_path + 'projects/PFAS/models/testing_Siddhas_chemistry/CH3F/with_Siddhas_thermo/chemkin/species_dictionary.txt'

#load siddas (her kinetics, RMG thermo)
species_siddhas, reactions_siddhas = load_chemkin_file(siddhas_chmkn, dictionary_path=siddhas_species, transport_path=siddhas_transport)


#load siddas (her kinetics, her thermo)
species_siddhas_thermo, reactions_siddhas_thermo = load_chemkin_file(siddhas_chmkn_with_thermo, dictionary_path=siddhas_species_with_thermo, transport_path=siddhas_transport_with_thermo)


In [4]:
#comparison of siddhas additions (with original thermo order) and previous CH3F model 
model_1_species_list=species_siddhas
model_2_species_list=species_siddhas_thermo

common_species_siddha, unique_species_siddhas, unique_species_siddhas_thermo = compare_species(model_1_species_list, model_2_species_list)

93 common species


In [5]:
print(len(common_species_siddha), len(unique_species_siddhas), len(unique_species_siddhas_thermo))
#model 2 previous CH3F, model 1 is siddhas

93 14 11


## Flipping

In [6]:
import cantera as ct
import cantera.ck2cti
import rmgpy.chemkin
#import numpy as np
import subprocess
import csv
#import scipy
import copy
import os

In [ ]:
#test for flipping

In [7]:
common_species_with_non_identical_thermo = []

non_identical_count=0
is_similar_count = 0 
is_different_count = 0
for i_kin, j_kin_thermo in common_species_siddha:
    if i_kin.thermo.is_identical_to(j_kin_thermo.thermo)==False:
        
        #pick them out
        common_species_with_non_identical_thermo.append((i_kin,j_kin_thermo))
        
        #increase the counts
        non_identical_count+=1
        if i_kin.thermo.is_similar_to(j_kin_thermo.thermo):
            is_similar_count+=1
        else: 
            is_different_count+=1
            
print(non_identical_count, is_similar_count, is_different_count)
#of the common species between the model, 17 common species possess thermos that are non-identical

17 11 6


In [8]:
#testing 
i_kin = common_species_with_non_identical_thermo[0][0]
j_kin_thermo = common_species_with_non_identical_thermo[0][1]
print(i_kin.thermo)
print(j_kin_thermo.thermo)

NASA(polynomials=[NASAPolynomial(coeffs=[4.20542,-0.00535566,2.51126e-05,-2.13766e-08,5.97538e-12,-10161.9,-0.921305], Tmin=(100,'K'), Tmax=(1084.11,'K')), NASAPolynomial(coeffs=[0.908224,0.0114542,-4.57178e-06,8.29201e-10,-5.66322e-14,-9719.96,13.9933], Tmin=(1084.11,'K'), Tmax=(5000,'K'))], Tmin=(100,'K'), Tmax=(5000,'K'), comment="""Thermo library: primaryThermoLibrary""")
NASA(polynomials=[NASAPolynomial(coeffs=[5.00702,-0.0126484,4.66821e-05,-4.59211e-08,1.57634e-11,-10222.4,-4.04227], Tmin=(200,'K'), Tmax=(1000,'K')), NASAPolynomial(coeffs=[1.68377,0.010013,-3.31268e-06,5.30234e-10,-3.13372e-14,-10018.8,9.71477], Tmin=(1000,'K'), Tmax=(6000,'K'))], Tmin=(200,'K'), Tmax=(6000,'K'), comment="""Thermo library: C1_C2_Fluorine""")


In [9]:
def switch_thermo(i_kin, j_kin_thermo):
    """ switch thermos and return i_kin """
    print(f'before:\n{i_kin.thermo}')
    i_kin.thermo = j_kin_thermo.thermo
    print(f'after:\n{i_kin.thermo}')
    return i_kin


In [10]:
#let's do the actual flipping now 
############################### one by one, lets flip the thermo of all the common species in between both of Siddhas models##########################################
count=0

for i_kin, j_kin_thermo in common_species_with_non_identical_thermo: 
    
    species_for_chemkin_file = []
    
    #iterate all the species in siddhas (her kinetics, RMG thermo) model
    for spc in species_siddhas: 
        
        #lets try and pick out the one species thermo we want to change, and just add the others
        if i_kin.is_isomorphic(spc): 
            
            #increase the count to make sure we covered all non-identical-thermo common species
            count+=1
            
            #let's do the actual switching here
            spc_with_switched_thermo = switch_thermo(spc, j_kin_thermo)
            #let's add it so its included in the chemkin file
            species_for_chemkin_file.append(spc_with_switched_thermo)
            
        else: 
            species_for_chemkin_file.append(spc)
            
    print(len(species_for_chemkin_file),len(species_siddhas))
    
    assert len(species_for_chemkin_file)==len(species_siddhas)
    
    #save the chemkin file 
    flipped_thermo_dir = '/work/westgroup/nora/Code/projects/PFAS/models/testing_Siddhas_chemistry/CH3F/flipping/thermo/flipped_models'


    chemkin_file = os.path.join(flipped_thermo_dir, f'{i_kin}.inp')
    rmgpy.chemkin.save_chemkin_file(chemkin_file, species_for_chemkin_file, reactions_siddhas, verbose=True, check_for_duplicates=True)

    #somehow, whhen it does chemkin.save_chemkin_file, its naming the species kind of weird
            
            
print(count)

before:
NASA(polynomials=[NASAPolynomial(coeffs=[4.20542,-0.00535566,2.51126e-05,-2.13766e-08,5.97538e-12,-10161.9,-0.921305], Tmin=(100,'K'), Tmax=(1084.11,'K')), NASAPolynomial(coeffs=[0.908224,0.0114542,-4.57178e-06,8.29201e-10,-5.66322e-14,-9719.96,13.9933], Tmin=(1084.11,'K'), Tmax=(5000,'K'))], Tmin=(100,'K'), Tmax=(5000,'K'), comment="""Thermo library: primaryThermoLibrary""")
after:
NASA(polynomials=[NASAPolynomial(coeffs=[5.00702,-0.0126484,4.66821e-05,-4.59211e-08,1.57634e-11,-10222.4,-4.04227], Tmin=(200,'K'), Tmax=(1000,'K')), NASAPolynomial(coeffs=[1.68377,0.010013,-3.31268e-06,5.30234e-10,-3.13372e-14,-10018.8,9.71477], Tmin=(1000,'K'), Tmax=(6000,'K'))], Tmin=(200,'K'), Tmax=(6000,'K'), comment="""Thermo library: C1_C2_Fluorine""")
107 107
before:
NASA(polynomials=[NASAPolynomial(coeffs=[2.5,9.05029e-14,-1.20495e-16,5.23432e-20,-6.99814e-24,29230.2,5.12616], Tmin=(100,'K'), Tmax=(4230.24,'K')), NASAPolynomial(coeffs=[2.5,3.58763e-09,-1.22015e-12,1.84118e-16,-1.04e-20,292

In [61]:
#fixing the trans.dat file 

path_to_transport_file = '/work/westgroup/nora/Code/projects/PFAS/models/testing_Siddhas_chemistry/CH3F/flipping/thermo/flipped_models/copies/tran.dat'

with open(path_to_transport_file, "r") as f:
    transport_lines = f.readlines()

    
indices =  {}
    
#traverse the old transport file    
for line in transport_lines: 
    match = re.search('(\S+)\(([0-9]+)\)', line)
    if match:
        to_be_replaced = match.group(1)
        index = match.group(2)
        indices[index] = [to_be_replaced]

#now find the new species names and change it 

print(len(indices))

  


105


In [62]:
with open('/work/westgroup/nora/Code/projects/PFAS/models/testing_Siddhas_chemistry/CH3F/flipping/thermo/flipped_models/species.txt','r') as f: 
    species_lines = f.readlines()
    
species_from_txt_file=[]
for i in species_lines: 
    match = re.search('(\S+\([0-9]+\))',str(i))
    if match: 
        species_from_txt_file.append(match.group(1))
        

In [63]:

for i in species_from_txt_file:
    for index in indices.keys():
        if f"({index})" in str(i):
            match = re.search('(\S+)\(([0-9]+)\)',str(i))
            print(match.group(1), match.group(2))
            assert match.group(2)==index
            list_ = indices[index]
            list_.append(match.group(1))
            indices[index] = list_


CH3F 1
O2 2
H2O 3
CH4 4
OH 5
H 6
O 7
H2 8
HO2 11
H2O2 12
CO 13
CO2 14
HCO 15
CH 17
CH2(T) 18
CH3 19
CH2O 20
HCCO 21
C2H 22
C2H2 23
H2CC 24
CH2(S) 25
CH3O 27
CH2CO 28
C2H3 29
C2H4 30
C2H5 32
CH2OH 33
CH3CO 34
CH2F 37
CH2F2 38
CHF2 39
CHF3 41
F 42
HF 43
FOH 44
OF 45
CHFO 46
FCO 47
CF2O 51
CHF 53
CF2 54
CF3O 55
CF 56
CH2CHF 61
CH2CF2 62
CH2F-CH2 67
CH3-CHF 68
CHF2-CH2 69
CH3-CF2 70
CH2F-CHF 71
CH2CF 75
CHFCH[Z] 76
CHFCF[Z] 77
CF2CH 78
CH2CFO 83
[O]OF 128
FCCF 130
C1OO1 133
[O]OCF 135
O-2 137
[O]C[O] 138
[CH]1OO1 139
CH2O2 140
CHO2 141
CO2 143
CHO2 144
[O]C([O])F 146
CFO2 152
[O]CF 161
FC1OO1 164
O[CH]F 165
[O]C(F)F 167
CHO3 178
CHFO2 180
CO3 207
CO3 215
C2O3 216
C2O3 221
C2HO3 246
CF2O2 279
CFO3 283
[O]C(O)(F)F 291
C2H2FO 314
CH2O3 391
C2HO3 422
[O]CO 448
FC1CO1 452
C2HO2 620
O[C](F)F 704
[O]C1(F)OO1 715
CO3 719
CHO3 769
[O]OC(F)F 780
C2HF2O 789
O[CH]O 846
C2O2 933
OC(F)(F)F 970
OO[C](F)F 1150
FC1(F)OO1 1390
C2H2O 2027
C2H2O 2622
C2O3 2860
[CH]C-2 3133
O2 3313


In [64]:
indices

{'1': ['CH3F', 'CH3F'],
 '2': ['O2', 'O2'],
 '3': ['H2O', 'H2O'],
 '4': ['CH4', 'CH4'],
 '5': ['OH', 'OH'],
 '25': ['CH2(S)', 'CH2(S)'],
 '43': ['HF', 'HF'],
 '37': ['CH2F', 'CH2F'],
 '18': ['CH2(T)', 'CH2(T)'],
 '8': ['H2', 'H2'],
 '53': ['CHF', 'CHF'],
 '11': ['HO2', 'HO2'],
 '133': ['C1OO1', 'C1OO1'],
 '138': ['[O]C[O]', '[O]C[O]'],
 '6': ['H', 'H'],
 '141': ['[O]C=O', 'CHO2'],
 '14': ['CO2', 'CO2'],
 '46': ['CHFO', 'CHFO'],
 '7': ['O', 'O'],
 '135': ['[O]OCF', '[O]OCF'],
 '13': ['CO', 'CO'],
 '164': ['FC1OO1', 'FC1OO1'],
 '146': ['[O]C([O])F', '[O]C([O])F'],
 '152': ['[O]C(=O)F', 'CFO2'],
 '180': ['O=C(O)F', 'CHFO2'],
 '20': ['CH2O', 'CH2O'],
 '15': ['HCO', 'HCO'],
 '161': ['[O]CF', '[O]CF'],
 '12': ['H2O2', 'H2O2'],
 '42': ['F', 'F'],
 '47': ['FCO', 'FCO'],
 '144': ['O=[C]O', 'CHO2'],
 '715': ['[O]C1(F)OO1', '[O]C1(F)OO1'],
 '19': ['CH3', 'CH3'],
 '56': ['CF', 'CF'],
 '283': ['[O]OC(=O)F', 'CFO3'],
 '178': ['[O]C(=O)O', 'CHO3'],
 '38': ['CH2F2', 'CH2F2'],
 '39': ['CHF2', 'CHF2'],


In [65]:
new_lines=[]

for line in transport_lines: 
    match = re.search('(\S+)\(([0-9]+)\)', line)
    if match:
        if match.group(2) in indices.keys():
            index = match.group(2)
            new_line = line.replace(indices[index][0],indices[index][1])
            new_lines.append(new_line)

with open('/work/westgroup/nora/Code/projects/PFAS/models/testing_Siddhas_chemistry/CH3F/flipping/thermo/flipped_models/tran_edited.dat','w') as f:
    f.writelines(new_lines)
        

In [71]:
x=os.listdir('/work/westgroup/nora/Code/projects/PFAS/models/testing_Siddhas_chemistry/CH3F/flipping/thermo/flipped_models/copies/')
y=[j for j in x if 'cti' in j]
' '.join(y)

'copy_12.cti copy_128.cti copy_161.cti copy_19.cti copy_3313.cti copy_37.cti copy_4.cti copy_42.cti copy_43.cti copy_44.cti copy_45.cti copy_67.cti copy_68.cti copy_7.cti copy_75.cti copy_76.cti copy_8.cti'